# Chroma DB

In [ ]:
import chromadb

chroma_client = chromadb.Client()

# collection은 일종의 rdb로 따지면 table과 같은 개념 

In [3]:
collection = chroma_client.create_collection(name="my_collection")  # collection 객체 생성 

In [4]:
# collection에 데이터 추가 
collection.add(
    documents=[
        'This is a document about pineapple',
        'This is a document about mango',
        'This is a document about strawberry'
    ],
    ids=['id1', 'id2', 'id3']   # 문장을 구분할 id를 지정
)

In [5]:
# query를 써서 질의를 해서 조회
results = collection.query(
    query_texts=['This is a query document about vietnam'], # 검색할 문장
    n_results=2  # 유사도를 가지고 있는 문장을 검색해서 반환할 결과의 개수 
)

In [6]:
results # embedding을 가지고 있지 않고 문자열을 자체로 가지고 있기 때문에 벡터값을 가지고 계산한 것은 아님. 따라서 몇개를 가져오든 순서를 기반으로 가지고 옴. 

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This is a document about pineapple',
   'This is a document about mango']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.2225853204727173, 1.2783520221710205]]}

### SciQ dataset 활용 ChromaDB 검색

In [7]:
# 데이터셋 로드 
from datasets import load_dataset

dataset = load_dataset("sciq", split="train")   # 학습용 데이터셋 로드 
dataset = dataset.filter(lambda x: x["support"] != "") # support가 빈값이 아닌 것에 대해서만 필터링 (공백인것 빼기) 

dataset # 특성 : 질문, 틀린답3, 틀린답1, 틀린답2, 정답, 추가적인 설명

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 10481
})

In [8]:
# chroma db 클라이언트 객체 및 콜렉션 생성 
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="sciq_support")

In [9]:
# !pip install huggingface_hub[hf_xet]

In [10]:
# 임베딩 모델 로드
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # 영어 임베딩하는데 가장 가벼운(가장 경량화된) 모델

In [11]:
supports = dataset["support"][:100]
support_embeddings = embedding_model.encode(supports).tolist()  # supports를 임베딩으로 변환, 리스트로 변환

In [12]:
len(support_embeddings[0])  # 384차원으로 임베딩되었다는 것을 볼 수 있다.

384

In [ ]:
collection.add(
    ids=[str(i)for i in range(0, 100)],
    embeddings=support_embeddings,
    metadatas=[{"type": "support", "text": text} for text in supports]  # 메타데이터 : 제공하고 있는 데이터에 대한 설명/ 데이터에 대해 인지하기 위한 데이터 / 직접 documents에 넣는 것보다 metadatas에 넣는 것이 훨씬 더 유연한 처리, 관리가 가능하다
)

In [14]:
questions = dataset["question"][:3]


In [15]:

question_embeddings = embedding_model.encode(questions).tolist()

results = collection.query(
    query_embeddings=question_embeddings,
    n_results=1
)

In [16]:
results

{'ids': [['36'], ['1'], ['2']],
 'embeddings': None,
 'documents': [[None], [None], [None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'type': 'support',
    'text': 'Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.'}],
  [{'text': 'Without Coriolis Effect the global winds would blow north to south or south to north. But Coriolis makes them blow northeast to southwest or the re

In [17]:
for i, q in enumerate(questions):
    print("Questions:", q)  # 현재 자연어로 된 질문
    print("Support:", results['metadatas'][i][0]['text'])   # 그에 대한 추가적인 설명 (support) n_results=1로 했기 때문에 1개만 가져옴(0번째 인덱스) -> 딕셔너리 형태니까 text를 통해 출력하게 만듬 
    print()

Questions: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Questions: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Support: Without Coriolis Effect the global winds would blow north to south or south to north

### Chroma DB를 활용한 키워드 기반 검색

In [18]:
# 샘플 데이터를 만듬 
documents = [
    "인공지능은 인간의 작업을 자동화하는 기술이다.",
    "기계 학습은 패턴을 학습하여 예측하는 기술이다.",
    "벡터 데이터베이스는 유사도를 기반으로 데이터를 검색하는 DB이다.",
    "딥러닝은 인공신경망을 활용한 기계 학습의 한 분야이다.",
    "강화 학습은 보상을 통해 최적의 행동을 학습하는 방법이다.",
    "자연어 처리 기술은 인간의 언어를 이해하고 생성할 수 있게 한다.",
    "AI는 이미지 인식, 음성 인식 등 다양한 분야에 활용된다.",
    "데이터 전처리는 모델 학습을 위한 필수 단계이다.",
    "오토인코더는 입력 데이터를 압축하고 복원하는 신경망이다.",
    "클러스터링은 데이터를 유사한 그룹으로 묶는 비지도 학습이다."
]

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# ChromaDB 클라이언트, 컬렉션 생성
client = chromadb.PersistentClient(path='./chroma_db')    # PersistentClient는 데이터를 디스크에 저장하는 클라이언트(해당 경로에 데이터베이스를 저장) / client는 메모리상에 저장됨 (메모리상에 저장된 데이터는 프로그램이 종료되면 데이터셋도 사라짐)
collection = client.get_or_create_collection(name='ai_documents')  # 컬렉션 생성 (없으면 생성, 있으면 가져옴)

# 텍스트 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')  # 경량화된 모델

In [20]:
for i, doc in enumerate(documents):
    embedding = model.encode(doc).tolist()
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{"text": doc}]
    )

In [21]:
query_keyword = 'AI'
query_embedding = model.encode(query_keyword).tolist()

results = collection.query(query_embeddings=query_embedding, n_results=2) 

for result in results['metadatas'][0]:
    print("검색된 문서:", result['text'])

검색된 문서: AI는 이미지 인식, 음성 인식 등 다양한 분야에 활용된다.
검색된 문서: 인공지능은 인간의 작업을 자동화하는 기술이다.


### 영화 추천 시스템

In [22]:
import pandas as pd
df = pd.read_csv('./data/tmdb_5000_movies.csv')
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [ ]:
# overview(줄거리) 임베딩 -> 저장
# 메타데이터 title 포함해서 저장
# 영화 제목을 입력 -> 유사한 영화 추천 
# title -> overview -> 임베딩 -> vector DB에 저장

In [24]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path = "./chromadb")
collection = client.get_or_create_collection(name='movies')

model = SentenceTransformer('all-MiniLM-L6-v2')

In [31]:
movies = [
    {
        "id": str(index),
        "title": row["title"],
        "overview": row["overview"] if pd.notna(row["overview"]) else ""
    } for index, row in df.iterrows()
]

In [33]:
for movie in movies:
    if movie["overview"]:
        overview_embedding = model.encode(movie["overview"]).tolist()
        collection.add(
            ids= [movie["id"]],
            embeddings= [overview_embedding],
            metadatas= [{"title":movie["title"], "text":movie["overview"]}]
        )

In [37]:
# 1. 제목입력 -> 줄거리를 찾고 -> 줄거리로 유사도 검색
input_title = "Inception" 
query_text = df.loc[df["title"] == input_title, "overview"].iloc[0]

query_embedding = model.encode(query_text).tolist()

results = collection.query(query_embeddings=[query_embedding], n_results=5)

for result in results['metadatas'][0]:
    print(result['title'])
    print(result['text'])
    print()

Identity Thief
When a mild-mannered businessman learns his identity has been stolen, he hits the road in an attempt to foil the thief -- a trip that puts him in the path of a deceptively harmless-looking woman.

The Master of Disguise
A sweet-natured Italian waiter named Pistachio Disguisey at his father Fabbrizio's restaurant, who happens to be a member of a family with supernatural skills of disguise. But moments later the patriarch of the Disguisey family is kidnapped Fabbrizio's former arch-enemy, Devlin Bowman, a criminal mastermind in an attempt to steal the world's most precious treasures from around the world. And it's up to Pistachio to track down Bowman and save his family before Bowman kills them!

The Death and Life of Bobby Z
A DEA agent provides former Marine Tim Kearney with a way out of his prison sentence: impersonate Bobby Z, a recently deceased drug dealer, in a hostage switch with a crime lord. When the negotiations go awry, Kearney flees, with Z's son in tow.

Abso

In [40]:
# 2. 원하는 줄거리 입력 -> 유사도 검색
# query_text = "a movie that infiltrating th target's unconsciousness"
query_text = "korea"

query_embedding = model.encode(query_text).tolist()

results = collection.query(query_embeddings=[query_embedding], n_results=5)

for result in results['metadatas'][0]:
    print(result['title'])
    print(result['text'])
    print()

Silmido
On 31 January 1968, 31 North Korean commandos infiltrated South Korea in a failed mission to assassinate President Park Chung-hee. In revenge, the South Korean military assembled a team of 31 criminals on the island of Silmido to kill Kim Il-sung for a suicide mission to redeem their honor, but was cancelled, leaving them frustrated. It is loosely based on a military uprising in the 1970s.

Tae Guk Gi: The Brotherhood of War
In 1950, in South Korea, shoe-shiner Jin-tae Lee and his 18-year-old old student brother, Jin-seok Lee, form a poor but happy family with their mother, Jin-tae's fiancé Young-shin Kim, and her young sisters. Jin-tae and his mother are tough workers, who sacrifice themselves to send Jin-seok to the university. When North Korea invades the South, the family escapes to a relative's house in the country, but along their journey, Jin-seok is forced to join the army to fight in the front, and Jin-tae enlists too to protect his young brother. The commander promise

### 논문 pdf 내용 검색

In [ ]:
# !pip install PyPDF2

In [58]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path='./chromadb')
# client.delete_collection('papers')  # 컬렉션 삭제
collection = client.get_or_create_collection(name='papers')

model = SentenceTransformer('all-MiniLM-L6-v2')

In [59]:
papers = [
    {"id": "1", "title":"딥러닝", "path":"./data/deep_learning.pdf"},
    {"id": "2", "title":"NLP", "path":"./data/nlp_paper.pdf"},
]

In [62]:
import PyPDF2

def extract_text_from_pdf(path):
    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = " ".join(page.extract_text() for page in reader.pages if page.extract_text())
    return text

In [63]:
for paper in papers:
    text = extract_text_from_pdf(paper["path"])
    embedding = model.encode(text).tolist()
    collection.add(
        ids=[paper["id"]],
        embeddings=[embedding],
        metadatas=[{'title':paper["title"]}],
        documents=[text]
    )

In [67]:
results = collection.get(
    include=['embeddings', 'documents', 'metadatas']
)

for emb in results['embeddings']:
    print(len(emb))

384
384


In [84]:
query_text = 'Natural Language'
query_embedding = model.encode(query_text).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)

results['metadatas'][0][0]['title']

'NLP'